<a href="https://colab.research.google.com/github/profteachkids/CHE2064_Spring2022/blob/main/Distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/che.py
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/FlattenWrap.ipynb
!pip install importnb

In [2]:
from importnb import Notebook
with Notebook(): 
    from FlattenWrap import DotDict, Range, RangeArray, Comp, CompArray, dtox

import che
import numpy as np
from scipy.optimize import root

In [3]:
p=che.Props(['Benzene','Toluene'])

In [4]:
d=DotDict()

d.P = 150e3
d.N = 5
d.NC = p.N_comps
d.F= 100.
d.Fz = np.array([0.4,0.6])
d.FT = 370.
d.D = 0.45 * d.F
d.B = d.F - d.D

d.F_stage = d.N // 2

d.reflux_ratio = 3.


In [5]:
def boiling_points(P):
    return root(lambda T: np.diagonal(p.Pvap(T)) - P, p.Tb).x

In [6]:
d.bp = boiling_points(d.P)
bp_scaled = (d.bp - np.mean(d.bp))/2 + np.mean(d.bp)
bp_minmax = np.min(d.bp), np.max(d.bp)

# Bug in root solver: convergence failure if T_guess is exactly linearly spaced
d.T = RangeArray(np.round(np.linspace(*bp_scaled,d.N)), *bp_minmax)
d.TC = Range(bp_scaled[-1], *bp_minmax)

Lguess = np.repeat(d.D*(d.reflux_ratio),d.N)
Lguess[d.F_stage:]+=d.F

d.L = RangeArray(Lguess, 0., 2*d.F*d.reflux_ratio)
d.V = RangeArray(np.repeat(d.D*(d.reflux_ratio+1),d.N), 0., 2*d.F*d.reflux_ratio)

QBGuess = d.D * d.reflux_ratio* np.mean(p.HvapNB)
d.QB = Range(QBGuess, 0., 2*QBGuess )
d.QC = Range(QBGuess, 0., 2*QBGuess)

d.Lx = CompArray(np.tile(d.Fz,(d.N,1)))
d.Vy = CompArray(np.tile(d.Fz,(d.N,1)))



In [7]:
wrap, x, d2, xtod, xtodunk  = dtox(d)

In [8]:
r = DotDict()

r.MB = np.zeros((d.N,d.NC))
zeros = np.zeros((1,p.N_comps))
def eqs(d):
    r.EQUIL = d.Lx* p.Pvap(d.T) - d.Vy*d.P


    r.Lin = np.r_[(d.L[0]*d.Vy[0])[None,:], d.L[1:,None]*d.Lx[:-1]]

    r.Lout = np.r_[d.L[1:,None]*d.Lx[:-1], (d.B*d.Lx[-1])[None,:]]

    r.Vin = np.r_[d.V[1:,None]*d.Vy[1:], zeros]
    r.Vout = d.V[:,None]*d.Vy

    r.MB = r.Lin - r.Lout + r.Vin - r.Vout

    r.MB[d.F_stage-1] += d.F*d.Fz

    r.TCBP = np.atleast_1d(np.sum(d.Vy[0]*p.Pvap(d.TC)) - d.P)

    r.T = np.insert(d.T,0,d.TC)
    r.EB = np.array(p.Hl(r.Lin, r.T[:-1]) + p.Hv(r.Vin, r.T[1:]) - p.Hl(r.Lout, r.T[1:]) - p.Hv(r.Vout, r.T[1:]))
    r.EB[d.F_stage-1] += np.array(p.Hl(d.F*d.Fz, d.FT))
    r.EB[-1]+=d.QB

    r.CONDENSER = np.atleast_1d(p.Hv(r.Vout[0],r.T[1]) - p.Hl(r.Vout[0], r.T[0]) - d.QC)

    r.RF = np.atleast_1d((d.V[0]-d.D)/d.D) - d.reflux_ratio
    return np.concatenate([np.ravel(r.EQUIL), np.ravel(r.MB), np.ravel(r.EB), r.TCBP, r.RF, r.CONDENSER]), r


In [9]:
eqs_wrapped = wrap(eqs)

In [10]:
eqs(d2)

(array([ 1.36290226e+04, -4.44234253e+04,  2.18992794e+04, -3.86459466e+04,
         3.08630714e+04, -3.23037764e+04,  4.05572950e+04, -2.53594243e+04,
         5.10195077e+04, -1.77742918e+04,  0.00000000e+00,  0.00000000e+00,
         1.42108547e-14,  1.42108547e-14,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -1.42108547e-14, -1.42108547e-14,
         3.82691382e+05, -2.29385386e+05, -1.62624519e+05, -1.63968263e+05,
        -1.46795334e+06,  3.47128435e+04,  4.44089210e-16,  9.69111749e+05]),
 {'CONDENSER': array([969111.74868589]),
  'EB': array([  382691.38171138,  -229385.38648117,  -162624.51867275,
          -163968.26272011, -1467953.34415918]),
  'EQUIL': DeviceArray([[ 13629.02255472, -44423.42527566],
               [ 21899.27940849, -38645.94657908],
               [ 30863.07144633, -32303.77635102],
               [ 40557.29499375, -25359.42434725],
               [ 51019.50772983, -17774.29177916]], dtype=float64),
  'Lin': array([[ 54., 

In [11]:
sol=root(eqs_wrapped, x)
print(sol)
solx=sol.x

    fjac: array([[-5.20070848e-02, -3.53759587e-02, -3.28592003e-05,
        -4.54043253e-06,  4.09892459e-06, -4.09812920e-06,
        -1.39161959e-04,  5.81292176e-05, -1.38295265e-03,
         5.57052023e-04,  7.03737110e-08, -6.99019315e-08,
        -1.20310216e-08,  1.03617258e-08,  2.08257870e-09,
        -2.61062201e-09,  1.39281919e-07, -1.40806801e-07,
         1.56328014e-07, -1.53077571e-07,  6.05016743e-01,
        -5.94381076e-01, -2.85326797e-04,  4.74111495e-03,
         5.27900266e-03,  1.54670196e-03, -2.75147313e-19,
        -5.25979672e-01],
       [-2.41901749e-02, -1.62700355e-02, -4.15228371e-02,
        -2.92272674e-02,  1.28690648e-05, -1.27355902e-05,
        -1.81639489e-05,  1.33906018e-05, -1.22171681e-03,
         5.14309384e-04,  1.81818755e-08, -1.76991741e-08,
        -3.66899501e-08,  3.13249604e-08,  5.67433662e-08,
        -5.50933159e-08, -3.86565251e-08,  3.66581900e-08,
         3.54021164e-07, -3.48790575e-07,  2.79457037e-01,
         5.03449873e

In [12]:
dunk=xtodunk(solx)
dunk

{'L': array([135.        , 134.95946008, 242.1728072 , 243.49778781,
        245.35629226]), 'Lx': array([[0.56855122, 0.43144878],
        [0.40492474, 0.59507526],
        [0.29401176, 0.70598824],
        [0.18860948, 0.81139052],
        [0.10574337, 0.89425663]]), 'QB': array(6212341.76385681), 'QC': array(5626081.44316109), 'T': array([377.18852199, 382.1982355 , 386.0070966 , 389.99483925,
        393.42068937]), 'TC': array(372.08931379), 'V': array([180.        , 179.95946008, 187.1728072 , 188.49778781,
        190.35629226]), 'Vy': array([[0.759647  , 0.240353  ],
        [0.61633592, 0.38366408],
        [0.49283802, 0.50716198],
        [0.34894483, 0.65105517],
        [0.21255214, 0.78744786]])}

In [13]:
eqs(xtod(sol.x,d2))

(array([-1.15026211e-04,  6.15012541e-05,  2.14263418e-05,  1.89218044e-05,
         3.81530845e-06, -3.83086444e-06,  6.18865597e-05, -2.55268387e-05,
         7.99126658e-04, -3.22834516e-04, -1.28907516e-08,  1.29581679e-08,
        -1.11924692e-10,  1.02120623e-09, -9.80577397e-10,  7.33393790e-10,
        -7.76454101e-08,  7.78061775e-08, -6.95502109e-08,  6.86599151e-08,
         1.08057261e-03, -2.66205985e-04,  2.96099111e-04, -2.63530947e-03,
        -4.22120746e-03,  4.32080182e-04,  0.00000000e+00, -1.75304897e-03]),
 {'CONDENSER': array([-0.00175305]),
  'EB': array([ 0.00108057, -0.00026621,  0.0002961 , -0.00263531, -0.00422121]),
  'EQUIL': DeviceArray([[-1.15026211e-04,  6.15012541e-05],
               [ 2.14263418e-05,  1.89218044e-05],
               [ 3.81530845e-06, -3.83086444e-06],
               [ 6.18865597e-05, -2.55268387e-05],
               [ 7.99126658e-04, -3.22834516e-04]], dtype=float64),
  'Lin': array([[102.55234496,  32.44765504],
         [ 76.731365